# Training logic

In [1]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

from torchvision import datasets, transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean=(0.5,), std=(0.5,))
                   ])),
    batch_size=batch_size,
    shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 106100546.24it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 35152551.89it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 27726449.17it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13444268.71it/s]


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(ResidualBlock, self).__init__()

        self.in_channel, self.out_channel = in_channel, out_channel

        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=1, padding=0)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(out_channel, out_channel, kernel_size=1, padding=0)

        if in_channel != out_channel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=1, padding=0)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = out + self.shortcut(x)
        return out

class ResNet(nn.Module):
    def __init__(self, color='gray'):
        super(ResNet, self).__init__()
        if color == "gray":
            self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        elif color == "rgb":
            self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)

        self.resblock1 = ResidualBlock(32, 64)
        self.resblock2 = ResidualBlock(64, 64)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [5]:
model = ResNet().to(device)

###  Learning Rate Scheduler

In [8]:
opt = optim.Adam(model.parameters(), lr = 0.03)

In [9]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

scheduler = ReduceLROnPlateau(opt, mode = 'min', verbose = True)
# epoch이 진행됨에 따라 학습률을 줄여 안정적으로 수렴하도록 도움을 줌

In [10]:
# training loop

def train_loop(dataloader, model, loss_fn, optimizer, scheduler, epoch):
  model.train()
  size = len(dataloader)

  for batch, (x, y) in enumerate (dataloader):
    x, y = x.to(device), y.to(device)

    pred = model(x)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if batch % 100 == 0:
      print(f"Epoch {epoch} : [{batch} / {size}] loss : {loss.item()}")

  scheduler.step(loss)
  return loss.item()

In [11]:
for epoch in range(10):
  loss = train_loop(train_loader, model, F.nll_loss, opt, scheduler, epoch)
  print(f"Epoch : {epoch} loss : {loss}")

Epoch 0 : [0 / 1875] loss : 2.315134286880493
Epoch 0 : [100 / 1875] loss : 1.7912001609802246
Epoch 0 : [200 / 1875] loss : 1.8850802183151245
Epoch 0 : [300 / 1875] loss : 1.6729724407196045
Epoch 0 : [400 / 1875] loss : 1.541942834854126
Epoch 0 : [500 / 1875] loss : 1.6807454824447632
Epoch 0 : [600 / 1875] loss : 1.395342469215393
Epoch 0 : [700 / 1875] loss : 1.4082295894622803
Epoch 0 : [800 / 1875] loss : 1.0983537435531616
Epoch 0 : [900 / 1875] loss : 1.1359412670135498
Epoch 0 : [1000 / 1875] loss : 1.2494008541107178
Epoch 0 : [1100 / 1875] loss : 1.25570547580719
Epoch 0 : [1200 / 1875] loss : 1.4497275352478027
Epoch 0 : [1300 / 1875] loss : 1.228243350982666
Epoch 0 : [1400 / 1875] loss : 1.230882167816162
Epoch 0 : [1500 / 1875] loss : 1.152808666229248
Epoch 0 : [1600 / 1875] loss : 1.6342580318450928
Epoch 0 : [1700 / 1875] loss : 1.128814458847046
Epoch 0 : [1800 / 1875] loss : 1.4296748638153076
Epoch : 0 loss : 0.824798583984375
Epoch 1 : [0 / 1875] loss : 1.216190

KeyboardInterrupt: ignored